# Suburb Venue Profiles and the Sydney Housing Market

### Parse CSV file of Sydney Suburb Names into Pandas Dataframe

In [1]:
# List of suburbs from "http://www.walksydneystreets.net/suburbssydneyall.htm"

In [2]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import geopy
!pip install folium
import folium


# The CSV file is uploaded to IBM Watson, which is accessed using my credentials. This is why the next cell is hidden.

     |████████████████████████████████| 92kB 14.5MB/s eta 0:00:01


In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
suburbs = df_data_0.T
suburbs.columns = ['Suburb Name']
suburbs.head()

,Suburb Name
0,Abbotsbury
1,Abbotsford
2,Acacia Gardens
3,Agnes Banks
4,Airds


### Get coordinates of each suburb using geocoder

locator = geopy.Nominatim(user_agent="Coursera_Capstone")

from geopy.extra.rate_limiter import RateLimiter

geocode = RateLimiter(locator.geocode, min_delay_seconds=2)

suburbs['location'] = suburbs['Address'].apply(geocode)

suburbs['point'] = suburbs['location'].apply(lambda loc: tuple(loc.point) if loc else None)

suburbs[['latitude', 'longitude', 'altitude']] = pd.DataFrame(suburbs['point'].tolist(), index=suburbs.index)

In [5]:
# The code was removed by Watson Studio for sharing.

In [6]:
suburbs.head()

,Suburb Name,location,point,latitude,longitude,altitude
0,Abbotsbury,"Abbotsbury, Sydney, Fairfield City Council, Ne...","(-33.8692846, 150.8667029, 0.0)",-33.869285,150.866703,0.0
1,Abbotsford,"Abbotsford, Sydney, City of Canada Bay Council...","(-33.8505529, 151.129759, 0.0)",-33.850553,151.129759,0.0
2,Acacia Gardens,"Acacia Gardens, Sydney, Blacktown City Council...","(-33.7324595, 150.9125321, 0.0)",-33.732459,150.912532,0.0
3,Agnes Banks,"Agnes Banks, Sydney, Hawkesbury City Council, ...","(-33.6145082, 150.7114482, 0.0)",-33.614508,150.711448,0.0
4,Airds,"Airds, Sydney, Campbelltown City Council, New ...","(-34.09, 150.8261111, 0.0)",-34.090000,150.826111,0.0


In [7]:
# Drop unneeded columns
suburbs = suburbs.drop(columns={"altitude", "point"})
suburbs.head()

,Suburb Name,location,latitude,longitude
0,Abbotsbury,"Abbotsbury, Sydney, Fairfield City Council, Ne...",-33.869285,150.866703
1,Abbotsford,"Abbotsford, Sydney, City of Canada Bay Council...",-33.850553,151.129759
2,Acacia Gardens,"Acacia Gardens, Sydney, Blacktown City Council...",-33.732459,150.912532
3,Agnes Banks,"Agnes Banks, Sydney, Hawkesbury City Council, ...",-33.614508,150.711448
4,Airds,"Airds, Sydney, Campbelltown City Council, New ...",-34.090000,150.826111


In [8]:
suburbs = suburbs.dropna()
suburbs.head()

,Suburb Name,location,latitude,longitude
0,Abbotsbury,"Abbotsbury, Sydney, Fairfield City Council, Ne...",-33.869285,150.866703
1,Abbotsford,"Abbotsford, Sydney, City of Canada Bay Council...",-33.850553,151.129759
2,Acacia Gardens,"Acacia Gardens, Sydney, Blacktown City Council...",-33.732459,150.912532
3,Agnes Banks,"Agnes Banks, Sydney, Hawkesbury City Council, ...",-33.614508,150.711448
4,Airds,"Airds, Sydney, Campbelltown City Council, New ...",-34.090000,150.826111


### Display map of Sydney with markers for each Suburb

In [9]:
map_syd = folium.Map(location=[-33.865143, 151.209900], zoom_start=10)

# add markers to map
for lat, lng, label in zip(suburbs['latitude'], suburbs['longitude'], suburbs['Suburb Name']):
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_syd)  
    
map_syd

### Retrieve venues in each suburb from FourSquare

In [10]:
# The code was removed by Watson Studio for sharing.

In [11]:
import requests

def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
sydney_venues = getNearbyVenues(names=suburbs['Suburb Name'],
                                   latitudes=suburbs['latitude'],
                                   longitudes=suburbs['longitude']
                                  )
print(sydney_venues.shape)

(6425, 7)


In [13]:
sydney_venues.head(20)

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbotsbury,-33.869285,150.866703,Abbotsbury Shops,-33.869554,150.865294,Convenience Store
1,Abbotsbury,-33.869285,150.866703,Stockdale Reserve,-33.871098,150.868060,Park
2,Abbotsbury,-33.869285,150.866703,817 bus stop,-33.869773,150.870122,Bus Station
3,Abbotsford,-33.850553,151.129759,The Cove Dining Co,-33.850933,151.129015,Café
4,Abbotsford,-33.850553,151.129759,The Cove Dining Co,-33.850950,151.129110,Café
5,Abbotsford,-33.850553,151.129759,Sistina Ristorante & Pizzeria,-33.848149,151.128307,Pizza Place
6,Abbotsford,-33.850553,151.129759,Lock Stock And Bottle,-33.847860,151.127970,Wine Shop
7,Abbotsford,-33.850553,151.129759,Cinnamon Thai Cuisine,-33.847697,151.128288,Thai Restaurant
8,Abbotsford,-33.850553,151.129759,Henry Lawson Park,-33.850350,151.125895,Park
9,Alexandria,-33.909157,151.192128,The Grounds Of Alexandria,-33.910774,151.194406,Café


### Develop Suburb Profiles
#### Count how many of each venue type are in each suburb - these counts will become the predictor variables for predicting the average house price of the suburb

In [14]:
profiles = sydney_venues.pivot_table(index='Suburb', columns='Venue Category', aggfunc='size', fill_value=0)

In [21]:
profiles.describe()

Venue Category,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Arepa Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
count,593.000000,593.000000,593.000000,593.000000,593.000000,593.000000,593.000000,593.000000,593.000000,593.000000,...,593.000000,593.000000,593.000000,593.000000,593.000000,593.000000,593.000000,593.000000,593.000000,593.000000
mean,0.001686,0.001686,0.001686,0.001686,0.013491,0.001686,0.001686,0.001686,0.008432,0.001686,...,0.006745,0.011804,0.148398,0.001686,0.010118,0.032040,0.020236,0.001686,0.006745,0.020236
std,0.041065,0.041065,0.041065,0.041065,0.129266,0.041065,0.041065,0.041065,0.091514,0.041065,...,0.081922,0.122732,0.857145,0.041065,0.100163,0.218994,0.140926,0.041065,0.081922,0.152442
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,2.000000,11.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000


### Retrieve the median house price of each suburb

In [79]:
import time
prices = pd.read_html("http://house.speakingsame.com/suburbtop.php?sta=nsw&cat=HomePrice&name=&page=0")[5]
for i in range(1, 42):
    url = "http://house.speakingsame.com/suburbtop.php?sta=nsw&cat=HomePrice&name=&page="+str(i)
    prices = prices.append(pd.read_html(url)[5])
    time.sleep(0.1)

prices.head()

,0,1,2
0,Rank,Suburb,Value
1,1,Point Piper,"$23,966,600"
2,2,Elizabeth Bay,"$8,881,200"
3,3,Watsons Bay,"$8,406,600"
4,4,Duffys Forest,"$7,328,300"


In [16]:
## todo plan:
# sum venue types ( training variables)
# get suburb price data (target variable)
# test train split
# train and evaluate a few models suitable for continuous target variables

In [80]:
prices.columns=prices.iloc[0]

In [81]:
prices.drop(prices.index[0], inplace=True)
prices.drop('Rank', axis=1,inplace=True)
prices

,Suburb,Value
1,Point Piper,"$23,966,600"
2,Elizabeth Bay,"$8,881,200"
3,Watsons Bay,"$8,406,600"
4,Duffys Forest,"$7,328,300"
5,Bellevue Hill,"$6,836,000"
6,Vaucluse,"$6,141,900"
7,Lavender Bay,"$5,624,700"
8,Rose Bay,"$5,336,400"
9,Sydney,"$5,330,000"
10,Darling Point,"$5,291,600"


In [86]:
profilesWithValue = pd.merge(profiles, prices, on=['Suburb'])

In [88]:
profilesWithValue.head()

,Suburb,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,...,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Value
0,Abbotsbury,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"$1,003,900"
1,Abbotsford,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,"$1,826,500"
2,Alexandria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"$1,532,700"
3,Alfords Point,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"$1,245,600"
4,Allambie Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"$1,664,600"
